# 한번 문장 유사도 기준으로 뽑아보자!
* 문장 임베딩 미리 학습된거로 하는게 조을듯
* countvectorzier 그냥 개수만 셈

In [1]:
# 필요 모듈 임포트
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import pandas.io.sql as sqlio
import psycopg
import numpy as np

In [2]:
# db에 있는거 바로 dataframe으로 바꿈
conn = psycopg.connect(host="sulleong.site", dbname="sulleong", user="sulleong", password="Sulleong104**", port=5432)
sql = "SELECT * FROM beer WHERE description IS NOT NULL;"
# sql = "select * from beer_merge_test;"
beers = sqlio.read_sql_query(sql, conn)
beers = beers.set_index('id')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_6508\4174560127.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  beers = sqlio.read_sql_query(sql, conn)


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

count_vect = CountVectorizer(stop_words='english')
matrix = count_vect.fit_transform(beers['description'])
matrix.shape

(1451, 9311)

In [4]:
# 코사인 유사도 계산
cosine_similar = cosine_similarity(matrix, matrix)

In [5]:
# index랑 원본 beer_id랑 달라서 idx 딕셔너리 새로 팜
idx = dict(zip(range(len(beers)), beers.index))

In [11]:
sentence_sim = []
for i in range(len(cosine_similar)):
    # index도 붙여줘야 하니까 enumerate 리스트 만듬
    similar_list = list(enumerate(cosine_similar[i]))
    similar_list = sorted(similar_list, key=lambda x:x[1], reverse=True)
    for j in range(1, 10):
        sentence_sim.append([idx[i], idx[similar_list[j][0]], similar_list[j][1]])
result_df = pd.DataFrame(sentence_sim, columns=["beer", "rec_beer", "distance"])
result_df.to_csv('./sentenc_sim.csv')

## 사전학습 모델 가져와서 문장 임베딩 하는것은 나중에 해봐야 할듯;;;

In [ ]:
# bert 사전학습 모델 가져옴
# import torch
# from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# for i in beers['description'][:5]:
#     marked_text = "[CLS]" + i + "[SEP]"

#     tokenized_text = tokenizer.tokenize(marked_text)
#     print(tokenized_text)